In [75]:
import os

os.getcwd()
#os.chdir('..')

'c:\\Users\\Dell\\Documents\\GitHub\\OCR-Invoice-Extraction'

In [53]:
import os
os.chdir("c:/Users/Dell/Documents/GitHub/OCR-Invoice-Extraction")


In [54]:
os.getcwd()

'c:\\Users\\Dell\\Documents\\GitHub\\OCR-Invoice-Extraction'

In [106]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate

import importlib
import utils.imgbase64
importlib.reload(utils.imgbase64)

from utils.imgbase64 import read_imgbase64

In [108]:
load_dotenv()

True

In [109]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    temperature = 0
)

## Structure Output

In [110]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional

class InvoiceItems(BaseModel):
    description : str = Field(..., description="Product's description in the invoice"),
    Taxes : Optional[float] = Field(..., description="Product's taxes")

class InvoiceData(BaseModel):
    buyer_name : str = Field(..., description="buyer's name in the invoice"),
    Invoice_number : str = Field(..., description="Invoice number")
    Invoice_date : str = Field(..., description= "Invoice date")
    items : List[InvoiceItems] = Field(..., description= "Invoice provided Items")

parser = PydanticOutputParser(pydantic_object=InvoiceData)

In [124]:
Examples = [
    {"invoice_img" : read_imgbase64(r"data\Invoices\2.jpg"),
     "invoice_data" : InvoiceData(
                        buyer_name = "Deco Addict",
                        Invoice_number = "FA04/2015/027662",
                        Invoice_date= "04-04-2015 ",
                        items= [
                            InvoiceItems(
                                description="Restaurant",
                                Taxes = 0.2
                            ),
                            InvoiceItems(
                                description= "Architecte Principal (Facturation sur Feuilles de Temps)",
                                Taxes = 0.2
                            ),
                            InvoiceItems(
                                description= "Armoire avec portes",
                                Taxes = 0.1
                            ),
                            InvoiceItems(
                                description= "Bureau personnalisable",
                                Taxes = 0.2
                            ),
                            InvoiceItems(
                                description= "Logiciel de conception de bureau",
                                Taxes = 0.2
                            ),
                            InvoiceItems(
                                description= "Bureau d'angle avec siège à gauche",
                                Taxes = 0.2
                            ),
                            InvoiceItems(
                                description= "Combinaison de bureau",
                                Taxes = 0.1
                            ),
                        ]

                    ).model_dump_json()


                    }
                    ]
                    
("assistant", "{invoice_data}"),

(('assistant', '{invoice_data}'),)

In [125]:
few_shot_example_prompt = ChatPromptTemplate.from_messages(
    [
        {

            "role": "user",
            "content" : [
                {
                    "type": "image",
                    "source_type": "base64",
                    "data": "{invoice_img}",
                    "mime_type": "image/jpeg",
                },
            ],
        },

        ("assistant", "{invoice_data}")
        

    ])

few_shot_prompts = FewShotChatMessagePromptTemplate(
    example_prompt = few_shot_example_prompt,
    examples = Examples
)

if __name__ == "__main__":
    for msg in few_shot_prompts.invoke({}).to_messages():
        msg.pretty_print()

================================ Human Message =================================

[{'type': 'image', 'source_type': 'base64', 'data': '...', 'mime_type': 'image/jpeg'}]
================================== Ai Message ==================================

{"buyer_name":"Deco Addict","Invoice_number":"FA04/2015/027662","Invoice_date":"04-04-2015 ","items":[{"description":"Restaurant","Taxes":0.2},{"description":"Architecte Principal (Facturation sur Feuilles de Temps)","Taxes":0.2},{"description":"Armoire avec portes","Taxes":0.1},{"description":"Bureau personnalisable","Taxes":0.2},{"description":"Logiciel de conception de bureau","Taxes":0.2},{"description":"Bureau d'angle avec siège à gauche","Taxes":0.2},{"description":"Combinaison de bureau","Taxes":0.1}]}


In [126]:
msgsTemplate = ChatPromptTemplate.from_messages([
    ("system", """
     Extract data from this Invoice
     follow this output schema {output_schema}
     """),
    few_shot_prompts,
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source_type": "base64",
                "data": "{image_data}",
                "mime_type": "image/jpeg",
            },
    ],
}
]).partial(output_schema = parser.get_format_instructions())


c:\Users\Dell\anaconda3\envs\OCR_Project\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description="buyer's name in the invoice"),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\Dell\anaconda3\envs\OCR_Project\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description="Product's description in the invoice"),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [127]:
pipeline = msgsTemplate | llm | parser

In [129]:


def get_invoice_data(img_path):
    print(f"Running inference on: {img_path}")
    return pipeline.invoke({
        "image_data": read_imgbase64(img_path)
    })

if __name__ == "__main__":
    result = get_invoice_data(r"data/Invoices/4.jpg")
    print(result)

Running inference on: data/Invoices/4.jpg
buyer_name='Biotech' Invoice_number='FA06/2015/088665' Invoice_date='2015-06-02' items=[InvoiceItems(description='Bureau personnalisable', Taxes=0.2), InvoiceItems(description='Virtual Home Staging', Taxes=None), InvoiceItems(description='Logiciel de conception de bureau', Taxes=0.2), InvoiceItems(description="Bureau d'angle avec siège à gauche", Taxes=0.2), InvoiceItems(description="Bureau d'angle avec siège à gauche", Taxes=0.2), InvoiceItems(description='Chaise de bureau', Taxes=0.2), InvoiceItems(description='Bureau pour quatre personnes', Taxes=0.2), InvoiceItems(description='Espace de travail individuel', Taxes=0.055), InvoiceItems(description='Logiciel de conception de bureau', Taxes=0.2)]
